## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# Create a connection to a Cassandra instance 

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace to do our work in
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Connect to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# -------------------------------
# Table Creation: music_app_history
# -------------------------------

# Create music_app_history Table
# The table is designed to facilitate efficient retrieval of music playback information by session.
# The PRIMARY KEY comprises 'sessionId' and 'itemInSession':
# - 'sessionId' serves as the partition key, meaning all items from the same session are stored together, 
#   which optimizes queries that retrieve all items from a specific session.
# - 'itemInSession' is used as a clustering key to maintain the order of playback within each session,
#   allowing for queries that need to access the exact order of songs in a session.

try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS music_app_history (
        sessionId int,
        itemInSession int,
        artist_name text,
        song_title text,
        song_length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
    """)
except Exception as e:
    print(e)


In [10]:
# -------------------------------
# Data Insertion into music_app_history Table
# Description: This section reads from 'event_datafile_new.csv', which contains event data from the music app.
# -------------------------------

# Open the CSV file and set up a reader
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Iterate over each line in the CSV file
    for line in csvreader:
        # Prepare the INSERT statement to add data into the Cassandra table
        # This INSERT statement maps CSV columns to Cassandra table columns
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Execute the query with actual data from the CSV
        # The data elements are type-casted as needed (e.g., sessionId and itemInSession are integers)
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Query to Retrieve Song Details by Session
query = """
SELECT artist_name, song_title, song_length 
FROM music_app_history 
WHERE sessionId = %s AND itemInSession = %s
"""

# Specify the values for the query parameters
session_id = 338
item_in_session = 4

try:
    rows = session.execute(query, (session_id, item_in_session))
    for row in rows:
        print(row.artist_name, row.song_title, row.song_length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
# -------------------------------
# Table Creation: user_session_history
# -------------------------------

# Define the table with appropriate data types and a primary key that includes a composite partition key (userId, sessionId)
# and a clustering column (itemInSession). The clustering order by itemInSession ascending ensures that data within a 
# partition is sorted by the order of item appearance within a session, facilitating queries that need to retrieve items 
# in the sequence they were played.
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_history (
        userId int,
        sessionId int,
        itemInSession int,
        artist_name text,
        song_title text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
    WITH CLUSTERING ORDER BY (itemInSession ASC);
    """)
except Exception as e:
    print(e)

# -------------------------------
# Data Insertion from CSV
# Description: Reads event data from a CSV file and inserts each row into the user_session_history table.
# -------------------------------
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Iterate over each line in the CSV file
    for line in csvreader:
        # Prepare the INSERT statement to add data into the Cassandra table
        # This INSERT statement maps CSV columns to Cassandra table columns
        query = "INSERT INTO user_session_history (userId, sessionId, itemInSession, artist_name, song_title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Execute the query with actual data from the CSV
        # The data elements are type-casted as needed (e.g., sessionId and itemInSession are integers)
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Query to retrieve artist, song title, and user's full name for a specific userId and sessionId, sorted by itemInSession
query = """
SELECT artist_name, song_title, user_first_name, user_last_name 
FROM user_session_history 
WHERE userId = %s AND sessionId = %s
"""

# Specify the values for the query parameters
user_id = 10
session_id = 182

try:
    rows = session.execute(query, (user_id, session_id))
    for row in rows:
        print(row.artist_name, row.song_title, row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
# -------------------------------
# Table Creation: song_history
# -------------------------------

# Create song_history Table
# The PRIMARY KEY is composed of 'song_title' and 'user_id'. 'song_title' is the partition key which groups all listens 
# of a song together, while 'user_id' as a clustering key ensures each entry is unique per user per song.
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_history (
        song_title text,
        user_id int,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY (song_title, user_id)
    )
    """)
except Exception as e:
    print(e)

# -------------------------------
# Data Insertion from CSV
# Description: Reads from a CSV file containing song play events and inserts each event into the song_history table.
# This is crucial for building up the database of song plays by different users.
# -------------------------------    
    
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Skip the header
    
    # Iterate over each line in the CSV file and prepare data for insertion
    for line in csvreader:
        query = "INSERT INTO song_history (song_title, user_id, user_first_name, user_last_name) "
        query += "VALUES (%s, %s, %s, %s)"
        
        # Execute the INSERT query with data mapped from the CSV file
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


# Query to fetch all users who have listened to a specified song
query = """
SELECT user_first_name, user_last_name 
FROM song_history 
WHERE song_title = %s
"""

# Specify the values for the query parameters
song_title = 'All Hands Against His Own'

try:
    rows = session.execute(query, [song_title])
    for row in rows:
        print(row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [14]:
# List of all tables
tables = ['music_app_history', 'user_session_history', 'song_history']

# # Loop through the list and attempt to drop each table
for table in tables:
    try:
        session.execute(f"DROP TABLE IF EXISTS {table};")
        print(f"Table {table} dropped successfully.")
    except Exception as e:
        print(f"Failed to drop table {table}: {e}")


Table music_app_history dropped successfully.
Table user_session_history dropped successfully.
Table song_history dropped successfully.


### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()